# Prob 2 Part 2 - exam

In [39]:
import numpy as np
import scipy as sp
import math

## Question 9

In [47]:
la = 20
mu = 7.5

Q = np.array([
    [-la, la,0,  0, 0],
    [mu, -la-mu,la,  0, 0],
    [0,2*mu, -la-2*mu, la, 0,],
    [0,0, 3*mu, -la-3*mu, la],
    [0,0,0,4*mu, -la-4*mu]
])
S = Q[0:5,0:5]
a = np.array([1,0,0,0,0])
def exp_matrix(A,t):
    return sp.linalg.expm(A*t)

expSt = exp_matrix(S,4)
a@expSt@np.ones(S.shape[0])
prob = 1-a@expSt@np.ones(S.shape[0])
prob

0.9991375651028974

### Question 10
use page 458

In [48]:
la = 20
mu = 7.5
s = 4
pi0_inv = 0
for j in range(s):
    pi0_inv += 1/math.factorial(j)*(la/mu)**j
pi0_inv += (la/mu)**s/(math.factorial(s)*(1-la/(s*mu)))
pi0 = 1/pi0_inv
L0 = pi0/s*(la/mu)**s*(la/(s*mu))/(1-la/(s*mu))**2

print(L0), print(pi0)

4.541019955654101
0.059866962305986704


(None, None)

### Question 11

In [49]:
def pi_k(k,la,mu,s,pi0):
    if k < s:
        return pi0*(la/mu)**k/math.factorial(k)
    else:
        return 1/math.factorial(s)*pi0*(la/mu)**s*(la/(s*mu))**(k-s)
pi1 = pi_k(1,la,mu,s,pi0)
states = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]
pis = [pi_k(k,la,mu,s,pi0) for k in states]
print(pis)

[0.059866962305986704, 0.15964523281596454, 0.21286031042128606, 0.1892091648189209, 0.12613944321261394, 0.08409296214174262, 0.05606197476116175, 0.037374649840774495, 0.024916433227182996, 0.01661095548478866, 0.01107397032319244, 0.007382646882128294, 0.0049217645880855285, 0.003281176392057019, 0.002187450928038013, 0.0014583006186920083, 0.0009722004124613388, 0.0006481336083075592, 0.00043208907220503945, 0.00028805938147002626, 0.00019203958764668416]


In [50]:
def prob_wait_time(i):
    if i <= s:
        return 0
    else:
        return (sp.stats.erlang.sf(10/60,i-s+1,0,1/30))*pi_k(i,la,mu,s,pi0)

In [51]:
accumulated_exp_wait_time = 0
for i in range(1000):
    accumulated_exp_wait_time += prob_wait_time(i)
accumulated_exp_wait_time

0.07062406925226981

In [52]:
import math

def erlang_c(lam, mu, s):
    """
    Erlang-C delay probability for an M/M/s queue.
    lam: arrival rate (per unit time)
    mu : service rate per server (per unit time)
    s  : number of servers
    """
    a = lam / mu
    rho = lam / (s * mu)
    if rho >= 1:
        raise ValueError("Unstable system: rho >= 1 (no steady state).")

    num = a**s / (math.factorial(s) * (1 - rho))
    den = sum(a**k / math.factorial(k) for k in range(s)) + num
    return num / den

In [53]:
lam = 20.0          # per hour
mu  = 7.5           # per hour  (mean 8 min)
s   = 4
t   = 10/60         # hours

a = lam/mu
rho = a/s

C = erlang_c(lam, mu, s)
P_wait = C * math.exp(-(s*mu - lam)*t)
print(P_wait)
P_wait = C * math.exp(-(s*mu - lam)*t)

0.07147399013513056


In [54]:
import math
import scipy.stats as st

def erlang_c(lam, mu, s):
    a = lam/mu
    rho = lam/(s*mu)
    if rho >= 1:
        raise ValueError("Unstable: rho >= 1")
    num = a**s / (math.factorial(s) * (1-rho))
    den = sum(a**k / math.factorial(k) for k in range(s)) + num
    return num/den

lam = 20.0
mu  = 7.5
s   = 4
t   = 10/60  # 10 minutes in hours

C = erlang_c(lam, mu, s)
P_tail_closed = C * math.exp(-(s*mu - lam)*t)
print("closed form:", P_tail_closed)

# sum check (needs pi0, pi_n)
a = lam/mu
rho = lam/(s*mu)
pi0 = 1/(sum(a**k/math.factorial(k) for k in range(s)) + a**s/(math.factorial(s)*(1-rho)))

def pi_n(n):
    if n < s:
        return pi0 * a**n / math.factorial(n)
    return pi0 * a**s / math.factorial(s) * rho**(n-s)

P_tail_sum = 0.0
for n in range(5000):
    if n >= s:
        k = n - s + 1
        P_tail_sum += st.erlang.sf(t, a=k, scale=1/(s*mu)) * pi_n(n)

print("sum:", P_tail_sum)

closed form: 0.07147399013513056
sum: 0.07147399013513056


# Question 12

In [84]:
# Rate of arrivals for (M/G/1) system
lam = 20.0
mu  = 7.5
s   = 4
rho = lam*1/mu #trafic intensity first system
rate_exit = 0
pi_aggregate = 0
for i in range(s):
    pi_aggregate += pi_n(i)
    rate_exit += pi_n(i)*mu*i
C = 1-(pi_aggregate)
rate_exit += C*s*mu
W_cc = 1/(mu*s-lam)*C
E_cc = 1/mu+W_cc
print("W_cc:", W_cc)
print("E_cc:", E_cc)

rate_enter = rate_exit*0.05
print("rate enter:", rate_enter)

la_cf = rate_enter
v_cf = 1/2 #mean -service distribution
tau_cf = 1/2 #sd - service distribution
rho_cf = la_cf*v_cf #trafic intensity
w = (la_cf*(tau_cf**2+v_cf**2))/(2*(1-rho_cf)) # waiting time
E_cf = w + v_cf
print("E_cf:", E_cf)
print("waiting time:", w)
#Total expected time in the system for customer that needs service in both systems
E_total = E_cc + E_cf
print("E_total:", E_total)



W_cc: 0.03784183296378418
E_cc: 0.17117516629711751
rate enter: 1.0
E_cf: 1.0
waiting time: 0.5
E_total: 1.1711751662971175
